# Pulling Amazon Product and Review Data

In [1]:
# Pulling Amazon Product and Review Data
# Open product file.
count = 0
 
loadedjson = open('meta_Clothing_Shoes_and_Jewelry.json', 'r')

In [2]:
allproducts = {}
 
listofcategories = {}

In [3]:
for aline in loadedjson:
    count += 1
    if count % 100000 == 0:
        print(count)
    aproduct = eval(aline)
    
    allproducts[aproduct['asin']] = aproduct
    
    for categories in aproduct['categories']:
        for acategory in categories:
            if acategory in listofcategories:
                listofcategories[acategory] += 1
            if acategory not in listofcategories:
                listofcategories[acategory] = 1                
  
listofcategories['Puma']

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000


7611

In [4]:
count = 0
allpumaasins = set()

In [5]:
for aproduct in allproducts:
   theproduct = allproducts[aproduct]
   count += 1
   if count % 10000 == 0:
        print(count/1503384)
   for categories in theproduct['categories']:
        for acategory in categories:
            if 'puma' in acategory.lower():
                allpumaasins.add(theproduct['asin'])

0.006651660520532346
0.013303321041064691
0.019954981561597036
0.026606642082129383
0.03325830260266173
0.03990996312319407
0.04656162364372642
0.053213284164258766
0.05986494468479111
0.06651660520532346
0.0731682657258558
0.07981992624638815
0.08647158676692049
0.09312324728745285
0.09977490780798519
0.10642656832851753
0.11307822884904987
0.11972988936958222
0.12638154989011458
0.13303321041064692
0.13968487093117926
0.1463365314517116
0.15298819197224395
0.1596398524927763
0.16629151301330863
0.17294317353384098
0.17959483405437332
0.1862464945749057
0.19289815509543803
0.19954981561597038
0.20620147613650272
0.21285313665703506
0.2195047971775674
0.22615645769809975
0.2328081182186321
0.23945977873916444
0.24611143925969678
0.25276309978022915
0.2594147603007615
0.26606642082129384
0.2727180813418262
0.2793697418623585
0.28602140238289087
0.2926730629034232
0.29932472342395555
0.3059763839444879
0.31262804446502024
0.3192797049855526
0.3259313655060849
0.33258302602661727
0.339234

In [6]:
# Write Puma ASINs file.
file = open('Michelle_Puglio-ASINS_puma.txt', 'w')
file.write(','.join(allpumaasins))
file.close()

In [7]:
# Open review file.
loadedjson = open('reviews_Clothing_Shoes_and_Jewelry.json', 'r')

In [8]:
allreviews = {}
count = 0

In [9]:
for aline in loadedjson:
    count += 1
    if count % 100000 == 0:
        print(count)
    areview = eval(aline)
    theasin = areview['asin']
    thereviewer = areview['reviewerID']
    
    if theasin in allpumaasins:
        thekey = '%s.%s' % (theasin,thereviewer)
        allreviews[thekey] = areview
        
len(allreviews)

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000


SyntaxError: EOL while scanning string literal (<string>, line 1)

In [10]:
# Write a JSON file of the Amazon Puma review data you'll use for topic 
# modeling.
import json
json.dump(allreviews, open('allpumareviews.json','w'))

# k-Means Topic Modeling (Worst Reviews)

In [11]:
import nltk
from nltk.corpus import stopwords
import sklearn
import sklearn.feature_extraction.text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

stop_words = stopwords.words('english')

stop_words.append('puma')


texts = set()
def load_texts(topicdata):
    for areview in topicdata:
        if 'reviewText' in topicdata[areview]:
            if int(topicdata[areview]['overall']) <= 1:
                reviewtext = topicdata[areview]['reviewText']
                summary = topicdata[areview]['summary']
                asin = topicdata[areview]['asin']
                review = '%s %s %s' % (asin, summary, reviewtext)
                texts.add(review)


print('loading texts')
load_texts(allreviews)

documents = list(texts)

vectorizer = TfidfVectorizer(stop_words=stop_words)
X = vectorizer.fit_transform(documents)

true_k = 20
 
model = KMeans(n_clusters=true_k, max_iter=100000)
model.fit(X)

print('Top terms per cluster')



order_centroids = model.cluster_centers_.argsort()[:,::-1]
terms = vectorizer.get_feature_names()

for i in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[i,:4]]
    print('%d: %s' % (i, ' '.join(topic_terms)))

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/_multiprocessing_helpers.py:38: UserWarning: [Errno 12] Cannot allocate memory.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


loading texts
Top terms per cluster
0: one star small baby
1: apart shoes shoe fell
2: money waste time shoes
3: wrong size ordered shoes
4: shoe 34 good size
5: watch time working water
6: quality poor bad price
7: tight send front shoes
8: size product large item
9: pair foot shoe shoes
10: shoes buy running bad
11: way small sizing size
12: good cheap color looks
13: size shoes small fit
14: narrow feet shoes shoe
15: small size half runs
16: uncomfortable shoe shoes feet
17: big size change thank
18: leather like made shoes
19: dont shirt sole buy


In [12]:
# Write a .txt file to save printed topics from your working Topic Model.
printed_topics = []
count = 0

for i in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[i,:4]]
    topic = '%d: %s' % (i, ' '.join(topic_terms))
    if topic not in printed_topics:
        printed_topics.append(topic)

file = open('1_Star_Printed_Topics.txt', 'w')
file.write('\n'.join(printed_topics))
file.close()

In [13]:
# Gensim in Python
import os
outfiles = {}

try:
    os.mkdir('output')
    
except OSError:
    print('Directory already exists!')
    
else:
     print ('Successfully created the directory.')
    

for atopic in range(true_k):
    topicterms = [terms[ind] for ind in order_centroids[atopic, :4]]
    outfiles[atopic] = open(os.path.join('output', '_'.join(topicterms) + '.txt'), 'w')

for areview in allreviews:
    if 'reviewText' in allreviews[areview]:
        areview = allreviews[areview]
        reviewwithmetadata = "%s %s %s" % (areview['asin'], areview['summary'], areview['reviewText'])
        Y = vectorizer.transform([reviewwithmetadata])
        
        for prediction in model.predict(Y):
            outfiles[prediction].write('%s\n' % reviewwithmetadata)

for n, f in outfiles.items():
    f.close()

Directory already exists!


# k-Means Topic Modeling (Best Reviews)

In [19]:
import nltk
from nltk.corpus import stopwords
import sklearn
import sklearn.feature_extraction.text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [20]:
stop_words = stopwords.words('english')

stop_words.append('puma')


texts = set()
def load_texts(topicdata):
    for areview in topicdata:
        if 'reviewText' in topicdata[areview]:
            if int(topicdata[areview]['overall']) >= 4:
                reviewtext = topicdata[areview]['reviewText']
                summary = topicdata[areview]['summary']
                asin = topicdata[areview]['asin']
                review = '%s %s %s' % (asin, summary, reviewtext)
                texts.add(review)


print('loading texts')
load_texts(allreviews)

documents = list(texts)

vectorizer = TfidfVectorizer(stop_words=stop_words)
X = vectorizer.fit_transform(documents)

true_k = 20
 
model = KMeans(n_clusters=true_k, max_iter=100000)
model.fit(X)

print('Top terms per cluster')



order_centroids = model.cluster_centers_.argsort()[:,::-1]
terms = vectorizer.get_feature_names()

for i in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[i,:4]]
    print('%d: %s' % (i, ' '.join(topic_terms)))

loading texts
Top terms per cluster
0: sneakers great comfortable love
1: great comfortable well fit
2: good quality product shoes
3: like shoes fit really
4: pair shoes another comfortable
5: shoe great comfortable good
6: bag gym b005lce58a great
7: nice shoes really comfortable
8: size small half shoes
9: stars five four great
10: loves son shoes old
11: love shoes color fit
12: watch great nice good
13: cute super daughter shoes
14: golf shoes shoe great
15: great shoes fit price
16: shoes comfortable great love
17: feet shoes narrow wide
18: running shoes great comfortable
19: muy de el la


In [21]:
# Write a .txt file to save printed topics from your working Topic Model.
printed_topics = []
count = 0

for i in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[i,:4]]
    topic = '%d: %s' % (i, ' '.join(topic_terms))
    if topic not in printed_topics:
        printed_topics.append(topic)

file = open('4_Star_Printed_Topics.txt', 'w')
file.write('\n'.join(printed_topics))
file.close()

In [22]:
# Gensim in Python
import os
outfiles = {}

try:
    os.mkdir('output')
    
except OSError:
    print('Directory already exists!')
    
else:
     print ('Successfully created the directory.')
    

for atopic in range(true_k):
    topicterms = [terms[ind] for ind in order_centroids[atopic, :4]]
    outfiles[atopic] = open(os.path.join('output', '_'.join(topicterms) + '.txt'), 'w')

for areview in allreviews:
    if 'reviewText' in allreviews[areview]:
        areview = allreviews[areview]
        reviewwithmetadata = "%s %s %s" % (areview['asin'], areview['summary'], areview['reviewText'])
        Y = vectorizer.transform([reviewwithmetadata])
        
        for prediction in model.predict(Y):
            outfiles[prediction].write('%s\n' % reviewwithmetadata)

for n, f in outfiles.items():
    f.close()

Directory already exists!
